# GISY6400 Capstone - Wildfire Hazard: Weather

Program: wildfire_weather.ipynb  
Programmer: Brian Gauthier  
Purpose: This notebook produces weather products for wildfire hazard mapping based on temperature, wind, humidity and precipitation. 
Date: May 14, 2025

### Import Python Modules

In [ ]:
import ee
import geemap

### Setup Work Space

In [ ]:
#ee.Authenticate()
ee.Initialize(project='your-earthengine-project-name')
geemap.ee_initialize()

In [ ]:
Map = geemap.Map()

### Define AOI & Seasons

In [ ]:
# Load HRM shapefile as Earth Engine FeatureCollection
hrm_shape_path = r"D:\Dropbox\COGS\Capstone\data\processed\centre_west.shp"
hrm_ee_fc = geemap.shp_to_ee(hrm_shape_path)

# Get geometry from the ee feature collection (since only geometry is needed for many spatial operations)
hrm_aoi = hrm_ee_fc.geometry()

# Add AOI to map for visualization
Map.addLayer(hrm_aoi, {}, 'HRM_AOI')

# Manually centre map on AOI centroid to avoid tuple errors
centroid = hrm_aoi.centroid(1).coordinates().getInfo()
Map.setCenter(centroid[0], centroid[1], zoom=10)

Map

In [ ]:
# Define seasons using dicts
seasons = {
    'spring': ('2024-04-01', '2024-06-15'),
    'summer': ('2024-06-16', '2024-08-31'),
    'fall':   ('2024-09-01', '2024-10-31')
}

### Function to Fetch Weather Layers by Season

In [ ]:
# Function to fetch weather layers per season, including wind speed calculation
def get_seasonal_weather(start, end, aoi):
    era5 = ee.ImageCollection("ECMWF/ERA5/HOURLY").filterDate(start, end).filterBounds(aoi)
    
    # Convert temperature bands from Kelvin to Celsius
    temp_c = era5.select('temperature_2m').map(lambda img: img.subtract(273.15))
    dewpoint_c = era5.select('dewpoint_temperature_2m').map(lambda img: img.subtract(273.15))

    # Max temp in Celsius
    temp = temp_c.max().clip(aoi).rename('temperature_2m')

    # Total precipitation (still in meters)
    precip = era5.select('total_precipitation').sum().clip(aoi).rename('total_precipitation')

    # Mean u and v components of wind at 10m
    u10 = era5.select('u_component_of_wind_10m').mean()
    v10 = era5.select('v_component_of_wind_10m').mean()

    # Calculate wind speed as hypotenuse of u and v components
    wind_speed = u10.hypot(v10).clip(aoi).rename('wind_speed')

    # VPD estimate (Temp - Dew Point) in Celsius
    t = temp_c.mean()
    td = dewpoint_c.mean()
    vpd = t.subtract(td).clip(aoi).rename('vpd')

    return {
        'temp': temp,
        'precip': precip,
        'wind': wind_speed,
        'vpd': vpd
    }

### Normalize Layers Function

In [ ]:
# Normalize layers 0-1
def normalize(image, min_val, max_val):
    return image.subtract(min_val).divide(max_val - min_val).clamp(0, 1)

### Seasonal Ranges

In [ ]:
# Season ranges
season_ranges = {
    'spring': {
        'temp': (19, 28),
        'vpd': (2.5, 5),
        'wind': (0.1, 0.5),
        'precip': (0.1, 0.15)
    },
    'summer': {
        'temp': (25, 35),
        'vpd': (1.5, 4),
        'wind': (1.5, 3.5),
        'precip': (0.1, 0.25)
    },
    'fall': {
        'temp': (23, 29),
        'vpd': (3.4, 4),
        'wind': (0.9, 1.3),
        'precip': (0.1, 0.15)
    }
}

### Fire Weather Hazard Index Function

In [ ]:
# Compute Fire Weather Hazard Index
def compute_fw_hi(season_key):
    start, end = seasons[season_key]
    layers = get_seasonal_weather(start, end, hrm_aoi)
    
    ranges = season_ranges[season_key]
    
    norm_temp = normalize(layers['temp'], *ranges['temp'])
    norm_vpd = normalize(layers['vpd'], *ranges['vpd'])
    norm_wind = normalize(layers['wind'], *ranges['wind'])
    norm_precip = normalize(layers['precip'], *ranges['precip'])
    
    fw_hi = (norm_temp.multiply(0.4)
             .add(norm_wind.multiply(0.3))
             .add(norm_precip.multiply(-0.2).add(0.2))
             .add(norm_vpd.multiply(0.1))
            ).clip(hrm_aoi)

    return fw_hi

### FWHI Classified

In [ ]:
# Fire Weather Hazard class labels
class_labels = {
    1: 'Low',
    2: 'Moderate',
    3: 'High',
    4: 'Extreme'
}

# Function for classified FWHI, Classify normalized FWHI into four categories: <=0.25: Low, <=0.5: Moderate, <=0.75: High, >0.75: Extreme

def classify_fw_hi(fw_hi_img):
    return (fw_hi_img
            .expression(
                "(b <= 0.25) ? 1 : (b <= 0.5) ? 2 : (b <= 0.75) ? 3 : 4",
                {'b': fw_hi_img}
            )
            .rename('fw_class')
            .clip(hrm_aoi))

### Visualize, Add to Map & Export

In [ ]:
# Visualization params
class_vis = {
    'min': 1,
    'max': 4,
    'palette': ['#a6cee3', '#1f78b4', '#fb9a99', '#e31a1c'],
    'format': 'png'
}

# Function to export Earth Engine image locally using geemap
def export_to_local(image, region, filename, scale=1000):
    print(f"Exporting {filename} ...")
    geemap.ee_export_image(
        image,
        filename,
        scale=scale,
        region=region,
        file_per_band=False
    )

# Define export folder path
export_folder = r"D:\Dropbox\COGS\Capstone\data\processed"
    
# Add layers to the map AND export classified images locally
for season in seasons.keys():
    fw = compute_fw_hi(season)
    classified = classify_fw_hi(fw)

    # Add to map
    Map.addLayer(classified, class_vis, f'FWHI Class {season.capitalize()}')

    # Build full path for export
    filename = f"{export_folder}\\FWHI_Classified_{season}.tif"
    export_to_local(classified, hrm_aoi, filename)

Map

### Get Stats

In [ ]:
# Helper for safe formatting
def safe_fmt(val):
    return f"{val:.2f}" if val is not None else "N/A"

# Print stats for each season
for season_key in seasons.keys():
    start, end = seasons[season_key]
    layers = get_seasonal_weather(start, end, hrm_aoi)

    temp_stats = layers['temp'].reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=hrm_aoi,
        scale=1000,
        maxPixels=1e9
    ).getInfo()

    vpd_stats = layers['vpd'].reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=hrm_aoi,
        scale=1000,
        maxPixels=1e9
    ).getInfo()

    wind_stats = layers['wind'].reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=hrm_aoi,
        scale=1000,
        maxPixels=1e9
    ).getInfo()

    precip_stats = layers['precip'].reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=hrm_aoi,
        scale=1000,
        maxPixels=1e9
    ).getInfo()

    print(f"\n=== {season_key.upper()} Stats ===")
    print(f"Temp Min: {safe_fmt(temp_stats.get('temperature_2m_min'))}, Max: {safe_fmt(temp_stats.get('temperature_2m_max'))}")
    print(f"VPD  Min: {safe_fmt(vpd_stats.get('vpd_min'))}, Max: {safe_fmt(vpd_stats.get('vpd_max'))}")
    print(f"Wind Min: {safe_fmt(wind_stats.get('wind_speed_min'))}, Max: {safe_fmt(wind_stats.get('wind_speed_max'))}")
    print(f"Precip Min: {safe_fmt(precip_stats.get('total_precipitation_min'))}, Max: {safe_fmt(precip_stats.get('total_precipitation_max'))}")

### Code for exporting unclassed rasters

In [ ]:
# Function to export Earth Engine image locally using geemap
def export_to_local(image, region, filename, scale=1000):
    print(f"Exporting {filename} ...")
    geemap.ee_export_image(
        image,
        filename,
        scale=scale,
        region=region,
        file_per_band=False
    )

export_folder = r"D:\Dropbox\COGS\Capstone\data\processed"

# --- SPRING ---
fw_spring = compute_fw_hi('spring')
Map.addLayer(fw_spring, {'min': 0, 'max': 2, 'palette': ['blue', 'yellow', 'red']}, 'FWHI Continuous Spring')
filename_spring = f"{export_folder}\\FWHI_Continuous_Spring.tif"
export_to_local(fw_spring, hrm_aoi, filename_spring)
print(f"Exported continuous FWHI raster for spring to:\n{filename_spring}")

# --- SUMMER ---
fw_summer = compute_fw_hi('summer')
Map.addLayer(fw_summer, {'min': 0, 'max': 2, 'palette': ['blue', 'yellow', 'red']}, 'FWHI Continuous Summer')
filename_summer = f"{export_folder}\\FWHI_Continuous_Summer.tif"
export_to_local(fw_summer, hrm_aoi, filename_summer)
print(f"Exported continuous FWHI raster for summer to:\n{filename_summer}")

# --- FALL ---
fw_fall = compute_fw_hi('fall')
Map.addLayer(fw_fall, {'min': 0, 'max': 2, 'palette': ['blue', 'yellow', 'red']}, 'FWHI Continuous Fall')
filename_fall = f"{export_folder}\\FWHI_Continuous_Fall.tif"
export_to_local(fw_fall, hrm_aoi, filename_fall)
print(f"Exported continuous FWHI raster for fall to:\n{filename_fall}")
